In [ ]:
import os
import pickle
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import geopandas as gpd

from scipy.stats import poisson
from statsmodels.graphics.tsaplots import plot_acf
from matplotlib.ticker import FormatStrFormatter

sys.path.append('..')

from src.models.lgcp_matern import LgcpMatern
from src.models.block_mixture_flat import FlatRegionMixture
from src.models.block_mixture_gp_softmax import BlockMixtureGpSoftmaxAllocation

from src.experiment.visualize import plot_traceplots
from src.experiment.model_management import build_lgcp_uid, build_block_mixture_flat_uid, build_block_mixture_gp_softmax_uid
from src.experiment.results_processing import process_mixture_based_model, process_block_mixtures, process_block_gps_mixtures
from src.experiment.results_processing import compute_t_test, process_lgcp, get_model_description, get_latest_snapshot_iteration
from src.experiment.printing_utilities import pretty_string, get_specification_table

from src.experiment.results_processing import compute_lgcp_predicted_counts_all_samples
from src.experiment.results_processing import compute_mixture_predicted_counts_all_samples
from src.experiment.results_processing import compute_rmses_for_samples
from src.experiment.results_processing import compute_avg_loglik

In [ ]:
%matplotlib inline

global_plot_format = "eps"

## Change this to the root folder of the repository
os.chdir(Path.home() / "workplace" / "spatial-poisson-mixtures")

results_dir_path = Path(os.getcwd()) / 'data' / 'results'
paper_output_dir_path = Path(os.getcwd()) / 'reports' / 'paper_plots'

A4_INCHES_X = 8.3
A4_INCHES_Y = 11.7

font = {'family' : 'normal',
        'size'   : 12}

plt.rc('font', **font)

### Latex header and footer files for tikz plots

In [ ]:
tex_fig_standalone_header = r"""
\documentclass{standalone}

% Language matters
\usepackage[T1]{fontenc}
\usepackage[utf8]{inputenc}
\usepackage[english]{babel}
\usepackage[autostyle=true]{csquotes} % Required to generate
                                      % language-dependent quotes in
                                      % the bibliography
\DeclareUnicodeCharacter{2212}{-} % Needed for minus sign

%% Tables
\usepackage{booktabs}
\usepackage{multirow}

%% Plots and graphics
\usepackage{contour}
\usepackage{tikz}

\usepackage{pgfplots}
\usepackage{pgfplotstable}
\pgfplotsset{compat=1.14}
\usepgfplotslibrary{statistics}

% Common lines for captions
\DeclareRobustCommand\full {\tikz[baseline=-0.5ex]\draw[very thick]
  (0,0)--(0.5,0);}
\DeclareRobustCommand\dotted{\tikz[baseline=-0.5ex]\draw[very
  thick,dotted] (0,0)--(0.54,0);}
\DeclareRobustCommand\dashed{\tikz[baseline=-0.5ex]\draw[very
  thick,dashed] (0,0)--(0.54,0);} \DeclareRobustCommand\chain
{\tikz[baseline=-0.5ex]\draw[very thick,dash dot dot] (0,0)--(0.5,0);}

% Colors and the lines for the performance plots
\definecolor{CustomBlue}{rgb}{0.12156862745098039, 0.4666666666666667,
  0.7058823529411765} \definecolor{CustomOrange}{rgb}{1.0,
  0.4980392156862745, 0.054901960784313725}
\definecolor{CustomGreen}{rgb}{0.17254901960784313,
  0.6274509803921569, 0.17254901960784313}
\definecolor{CustomRed}{rgb}{0.8392156862745098, 0.15294117647058825,
  0.1568627450980392)} \DeclareRobustCommand\LineBlue
{\tikz[baseline=-0.6ex]\draw[very thick, CustomBlue] (0,0)--(0.5,0);}
\DeclareRobustCommand\LineOrange {\tikz[baseline=-0.6ex]\draw[very
  thick, CustomOrange] (0,0)--(0.5,0);}
\DeclareRobustCommand\LineGreen {\tikz[baseline=-0.6ex]\draw[very
  thick, CustomGreen] (0,0)--(0.5,0);} \DeclareRobustCommand\LineRed
{\tikz[baseline=-0.6ex]\draw[very thick, CustomRed] (0,0)--(0.5,0);}

% Colors and the lines for the mixture plots
\definecolor{CustomK1}{rgb}{0.9312692223325372, 0.8201921796082118,
  0.7971480974663592}
\definecolor{CustomK2}{rgb}{0.8707745887378947,
  0.671185774711061, 0.6918315421885759}
\definecolor{CustomK3}{rgb}{0.7840440880599453, 0.5292660544265891,
  0.6200568926941761}
\definecolor{CustomK4}{rgb}{0.6672565752652589,
  0.40671838146419587, 0.5620016466433286}
\definecolor{CustomK5}{rgb}{0.5151069036855755, 0.29801047535056074,
  0.49050619139300705}
\definecolor{CustomK6}{rgb}{0.34417698402907926, 0.2047057228850617,
  0.3876123154962551}
\definecolor{CustomK7}{rgb}{0.1750865648952205,
  0.11840023306916837, 0.24215989137836502}
\DeclareRobustCommand\LineOne {\tikz[baseline=-0.6ex]\draw[very thick,
  CustomK1] (0,0)--(0.5,0);}
\DeclareRobustCommand\LineTwo
{\tikz[baseline=-0.6ex]\draw[very thick, CustomK2] (0,0)--(0.5,0);}
\DeclareRobustCommand\LineThree {\tikz[baseline=-0.6ex]\draw[very
  thick, CustomK3] (0,0)--(0.5,0);}
\DeclareRobustCommand\LineFour
{\tikz[baseline=-0.6ex]\draw[very thick, CustomK4] (0,0)--(0.5,0);}
\DeclareRobustCommand\LineFive {\tikz[baseline=-0.6ex]\draw[very
  thick, CustomK5] (0,0)--(0.5,0);}
\DeclareRobustCommand\LineSix
{\tikz[baseline=-0.6ex]\draw[very thick, CustomK6] (0,0)--(0.5,0);}
\DeclareRobustCommand\LineSeven {\tikz[baseline=-0.6ex]\draw[very
  thick, CustomK7] (0,0)--(0.5,0);}


%% Macros for text that needs to be consistent stylewise and appears often.
\DeclareRobustCommand{\CovEffect}{\texttt{IMP}}

% Import macros for writing mathematics
\input{../defs.tex}

\begin{document}
"""

tex_fig_standalone_footer = r"""
\end{document}
"""

# Specification table

In [ ]:
specification_table_df = get_specification_table()
# print(specification_table_df.to_latex())
specification_table_df

# Analysis code launcher

These cells need to be run before any of the analysis/plotting code below is run. This will run all the possible
combinations of the experiments that are specified in the below as parameters of `process_lgcp`, `process_block_mixtures`,
`process_block_gps_mixtures` functions. Note that to be able to run this analysis, corresponding experiments must be
launched as described in the `README.md` file in the root folder of the repository.

In [ ]:
lgcp_uid_prefix = "2020-02-12-LGCP-MATERN"
lgcp_results_analysis_tag = 'lgcp_analysis_final'

blockmix_uid_prefix = "2020-02-18-BLOCKMIX"
blockmix_results_analysis_tag = 'blockmix_analysis_alpha1_over_K_final'

blockgp_softmax_uid_prefix = "2020-01-27-BLOCKMIX-GP-SOFTMAX"
blockgp_softmax_results_analysis_tag = 'blockgp_softmax_analysis_S500'

## LGCP

Process results of different combinations of the LGCP model. We test 4 types of model specifications for the covariates
and we test it on two different time periods.

In [ ]:
process_lgcp(uid_prefix=lgcp_uid_prefix,
             chain_no="1",
             set_c_types=['burglary'],
             set_resolutions=["400"],
             set_model_names=['burglary_raw_1', 'burglary_raw_2', 'burglary_raw_3', 'burglary_raw_4'],
             set_t_periods=['12015-122015', '12013-122015'],
             output_name=lgcp_results_analysis_tag,
             S=5_000,
             pool_size=10,
             max_paipei_n=500)

## BLOCK MIXTURE

Process results for the SAM-GLM model with independent blocks.

In [ ]:
process_block_mixtures(uid_prefix=blockmix_uid_prefix,
                       chain_no="1",
                       set_c_types=['burglary'],
                       set_resolutions=["400"],
                       set_model_names=['burglary_raw_1', 'burglary_raw_2', 'burglary_raw_3', 'burglary_raw_4'],
                       set_t_periods=['12015-122015', '12013-122015'],
                       set_block_schemes=["msoa"],
                       set_Ks=[1, 2, 3, 4, 5, 6, 7, 8],
                       output_name=blockmix_results_analysis_tag,
                       S=5_000,
                       pool_size=10,
                       max_paipei_n=500)

## BLOCK GP MIXTURE

In [ ]:
process_block_gps_mixtures(uid_prefix=blockgp_softmax_uid_prefix,
                          chain_no=None,
                          set_c_types=['burglary'],
                          set_resolutions=[400],
                          set_model_names=['burglary_raw_4'],
                          set_t_periods=['12015-122015', '12013-122015'],
                          set_block_schemes=['msoa'],
                          set_Ks=[2,3,4],
                          set_lengthscales=["500", "1000", "1500", "2000"],
                          output_name=blockgp_softmax_results_analysis_tag,
                          S=5_000,
                          pool_size=10,
                          max_paipei_n=500)

# Performance tables

## LGCP

In [ ]:
lgcp_main_results_df = pd.read_csv(results_dir_path / f'lgcp_model_results_{lgcp_results_analysis_tag}.csv')
lgcp_main_results_df

## SAM-GLM Independent Blocks

In [ ]:
blockmix_main_results_df = pd.read_csv(results_dir_path / f'block_mixtures_model_results_{blockmix_results_analysis_tag}.csv')
blockmix_main_results_df

## SAM-GLM Dependent Blocks

In [ ]:
blockmix_main_results_df = pd.read_csv(results_dir_path / f'block_mixtures_gp_model_results_{blockgp_softmax_results_analysis_tag}.csv')
blockmix_main_results_df

# Comparative Plots LGCP vs SAM-GLM with dependent blocks vs SAM-GLM with independent blocks

## SAM-GLM Dependence vs Independence analysis

In [ ]:
t_period = '12015-122015'
model_spec = 'burglary_raw_4'
# K = 3

S = 5_000

for K in [2,3,4]:

    print("\n")
    print(f"------------------------- K={K} -------------------------")
    print("\n")

    set_1_uid = build_block_mixture_flat_uid(prefix=blockmix_uid_prefix, chain_no="1", block_scheme="msoa",
                                             c_type="burglary", t_period=t_period, model_spec=model_spec, resolution=400, K=K)

    set_2_uid = build_block_mixture_gp_softmax_uid(prefix=blockgp_softmax_uid_prefix, chain_no=None, block_scheme="msoa",
                                                 c_type="burglary", t_period=t_period, model_spec=model_spec, resolution=400, K=K, lengthscale=1500)

    results_1 = process_mixture_based_model((FlatRegionMixture, set_1_uid, S))
    results_2 = process_mixture_based_model((BlockMixtureGpSoftmaxAllocation, set_2_uid, S))
    
    print(f"K = {K}")
    
    print("Dependent vs Independent (log-likelihood)")
    t_stat, p_val = compute_t_test(results_1['AVG_LOGLIK_OUT'], results_2['AVG_LOGLIK_OUT'])
    print(f"M1: {np.mean(results_1['AVG_LOGLIK_OUT'])}, S1: {np.std(results_1['AVG_LOGLIK_OUT'])}, M2: {np.mean(results_2['AVG_LOGLIK_OUT'])}, S2: {np.std(results_2['AVG_LOGLIK_OUT'])}")
    print(f"t-statistic {t_stat}, P-value: {p_val}")
    
    print("Dependent vs Independent (RMSE)")
    t_stat, p_val = compute_t_test(results_1['RMSE_OUT'], results_2['RMSE_OUT'])
    print(f"M1: {np.mean(results_1['RMSE_OUT'])}, S1: {np.std(results_1['RMSE_OUT'])}, M2: {np.mean(results_2['RMSE_OUT'])}, S2: {np.std(results_2['RMSE_OUT'])}")
    print(f"t-statistic {t_stat}, P-value: {p_val}")



## Performance: Held-out log-likelihood and RMSE

Generate plots for all model specifications and for both LGCP and SAM-GLM model  (K=1,...,5). The code below generates
tikz picture which can be rendered using LaTeX using TIKz library.

In [ ]:
S = 5_000
c_type = 'burglary'

left_min = 0.05
right_min = 0.95

for t_period in ['12015-122015', '12013-122015']:

    model_spec_color_map = {'burglary_raw_1': 'CustomBlue', 
                 'burglary_raw_2': 'CustomOrange', 
                 'burglary_raw_3': 'CustomGreen',
                 'burglary_raw_4': 'CustomRed'}


    full_plot_str = f"""% This is a computer-generated file. DO NOT EDIT."""
    full_plot_str += tex_fig_standalone_header
    full_plot_str += f"""\\begin{{tikzpicture}}%
\\begin{{axis}}[width=0.49\\textwidth, title={{Held-out log-likelihood}}, xlabel={{$K$}},xmin=0.6, xmax=8.4, xtick={{1,...,8}}, boxplot/draw direction=y]

"""

    for spec_idx, model_spec in enumerate(['burglary_raw_1', 'burglary_raw_2', 'burglary_raw_3', 'burglary_raw_4']):
        for K in [1,2,3,4,5,6,7,8]:        
            uid = build_block_mixture_flat_uid(prefix=blockmix_uid_prefix, chain_no="1", block_scheme="msoa",
                                                 c_type=c_type, t_period=t_period, model_spec=model_spec, resolution=400, K=K)

            context, K = get_model_description(uid)
            model = FlatRegionMixture(uid=uid, grid_context=context, K=K)

            iteration_no = get_latest_snapshot_iteration(uid)
            samples_tuple = model.load_samples_snapshot(iteration_no)

            beta_flat_samples = samples_tuple[0][-S:, :]
            Z_flat_samples = samples_tuple[1][-S:, :]

            S = min(beta_flat_samples.shape[0], Z_flat_samples.shape[0])
            print(f"Number of samples: {S}, UID: {uid}")

            beta_matrix_samples = beta_flat_samples.reshape((S, context.J, model.K), order='F')
            Z_matrix_samples = np.zeros((S, context.N, K), dtype=bool)
            Z_matrix_samples[np.repeat(range(S), context.N),
                             np.tile(range(context.N), S),
                             Z_flat_samples.flatten(order='C')] = 1


            counts_predicted_samples = compute_mixture_predicted_counts_all_samples(Z_matrix_samples,
                                                                                    beta_matrix_samples,
                                                                                    context.covariates)
            metric_all = compute_avg_loglik(counts_predicted_samples, context.test_counts)
            full_plot_str += f"\\addplot[{model_spec_color_map[model_spec]}, boxplot prepared={{draw position={K - 0.2125 + spec_idx * 0.125}, box extend=0.075, lower whisker={np.min(metric_all)},lower quartile={np.quantile(metric_all, left_min)},median={np.median(metric_all)},upper quartile={np.quantile(metric_all, right_min)},upper whisker={np.max(metric_all)}}}] coordinates{{}};\n"

        uid = build_lgcp_uid(prefix=lgcp_uid_prefix,
                                 chain_no="1",
                                 c_type=c_type,
                                 t_period=t_period,
                                 model_spec=model_spec,
                                 resolution="400")
        context = get_model_description(uid)
        model = LgcpMatern(uid=uid, grid_context=context)

        iteration_no = get_latest_snapshot_iteration(uid)
        samples = model.load_samples_snapshot(iteration_no)
        S = min(S, samples.shape[0])
        print(f"Number of samples: {S}, UID: {uid}")

        beta_samples = samples[-S:, model.NN:(model.NN + context.J)]
        f_samples = samples[-S:, :model.NN][:, context.mask]

        counts_predicted_samples = compute_lgcp_predicted_counts_all_samples(beta_samples, f_samples, context.covariates)

        metric_all = compute_avg_loglik(counts_predicted_samples, context.test_counts)
        full_plot_str +=  f"\\addplot[thick, dotted, {model_spec_color_map[model_spec]}, boxplot prepared={{draw position={2 * (spec_idx) + 1.5}, box extend=0.25, lower whisker={np.min(metric_all)},lower quartile={np.quantile(metric_all, left_min)},median={np.median(metric_all)},upper quartile={np.quantile(metric_all, right_min)},upper whisker={np.max(metric_all)}}}] coordinates{{}};\n"
    full_plot_str += f"""\end{{axis}}%
\end{{tikzpicture}}%"""
    full_plot_str += tex_fig_standalone_footer
    with open(paper_output_dir_path / f'blockmix_vs_lgcp_loglikout_{c_type}_msoa_{t_period}.tex', "w") as text_file:
        print(full_plot_str, file=text_file)

In [ ]:
S = 5_000
c_type = 'burglary'

left_min = 0.05
right_min = 0.95

model_spec_color_map = {'burglary_raw_1': 'CustomBlue', 
             'burglary_raw_2': 'CustomOrange', 
             'burglary_raw_3': 'CustomGreen',
             'burglary_raw_4': 'CustomRed'}

for t_period in ['12015-122015', '12013-122015']:
    full_plot_str = """% This is a computer-generated file. DO NOT EDIT."""
    full_plot_str += tex_fig_standalone_header
    full_plot_str += f"""\\begin{{tikzpicture}}%
\\begin{{semilogyaxis}}[width=0.49\\textwidth, title={{RMSE}}, xlabel={{$K$}},xmin=0.6, xmax=8.4, xtick={{1,...,8}}, boxplot/draw direction=y]\n"""

    for spec_idx, model_spec in enumerate(['burglary_raw_1', 'burglary_raw_2', 'burglary_raw_3', 'burglary_raw_4']):
        for K in [1,2,3,4,5,6,7,8]:        
            uid = build_block_mixture_flat_uid(prefix=blockmix_uid_prefix, chain_no="1", block_scheme="msoa",
                                                 c_type=c_type, t_period=t_period, model_spec=model_spec, resolution=400, K=K)

            context, K = get_model_description(uid)
            model = FlatRegionMixture(uid=uid, grid_context=context, K=K)

            iteration_no = get_latest_snapshot_iteration(uid)
            samples_tuple = model.load_samples_snapshot(iteration_no)

            beta_flat_samples = samples_tuple[0][-S:, :]
            Z_flat_samples = samples_tuple[1][-S:, :]

            S = min(beta_flat_samples.shape[0], Z_flat_samples.shape[0])
            print(f"Number of samples: {S}, UID: {uid}")

            beta_matrix_samples = beta_flat_samples.reshape((S, context.J, model.K), order='F')
            Z_matrix_samples = np.zeros((S, context.N, K), dtype=bool)
            Z_matrix_samples[np.repeat(range(S), context.N),
                             np.tile(range(context.N), S),
                             Z_flat_samples.flatten(order='C')] = 1


            counts_predicted_samples = compute_mixture_predicted_counts_all_samples(Z_matrix_samples,
                                                                                    beta_matrix_samples,
                                                                                    context.covariates)

            metric_all = compute_rmses_for_samples(counts_predicted_samples, context.test_counts)
            full_plot_str += f"\\addplot[{model_spec_color_map[model_spec]}, boxplot prepared={{draw position={K - 0.2125 + spec_idx * 0.125}, box extend=0.075, lower whisker={np.min(metric_all)},lower quartile={np.quantile(metric_all, left_min)},median={np.median(metric_all)},upper quartile={np.quantile(metric_all, right_min)},upper whisker={np.max(metric_all)}}}] coordinates{{}};\n"

        uid = build_lgcp_uid(prefix=lgcp_uid_prefix,
                                 chain_no="1",
                                 c_type=c_type,
                                 t_period=t_period,
                                 model_spec=model_spec,
                                 resolution="400")
        context = get_model_description(uid)
        model = LgcpMatern(uid=uid, grid_context=context)

        iteration_no = get_latest_snapshot_iteration(uid)
        samples = model.load_samples_snapshot(iteration_no)
        S = min(S, samples.shape[0])
        print(f"Number of samples: {S}, UID: {uid}")

        beta_samples = samples[-S:, model.NN:(model.NN + context.J)]
        f_samples = samples[-S:, :model.NN][:, context.mask]

        counts_predicted_samples = compute_lgcp_predicted_counts_all_samples(beta_samples, f_samples, context.covariates)

        metric_all = compute_rmses_for_samples(counts_predicted_samples, context.test_counts)
        full_plot_str +=  f"\\addplot[thick, dotted, {model_spec_color_map[model_spec]}, boxplot prepared={{draw position={2 * (spec_idx) + 1.5}, box extend=0.25, lower whisker={np.min(metric_all)},lower quartile={np.quantile(metric_all, left_min)},median={np.median(metric_all)},upper quartile={np.quantile(metric_all, right_min)},upper whisker={np.max(metric_all)}}}] coordinates{{}};\n"
    full_plot_str += f"""\end{{semilogyaxis}}%
\end{{tikzpicture}}%"""
    full_plot_str += tex_fig_standalone_footer
    with open(paper_output_dir_path / f'blockmix_vs_lgcp_rmseout_{c_type}_msoa_{t_period}.tex', "w") as text_file:
        print(full_plot_str, file=text_file)

## PAI / PEI
Compute the PAI/PEI scores and store in the '.dat' file to be plotted separately, e.g. using gnuplot.


In [ ]:
mixture_tag = blockmix_results_analysis_tag
lgcp_tag = lgcp_results_analysis_tag

chosen_model = 'burglary_raw_4'
chosen_block_scheme = 'msoa'
chosen_crime_type = 'burglary'

max_K = 7
max_n = 500

for chosen_statistic in ['PAI', 'PEI']:
    for chosen_time_period in ['12015-122015', '12013-122015']:

        glm_mix_pai_pei_results_df = pd.read_csv(results_dir_path / f'block_mixtures_pai_pei_df_{mixture_tag}.csv')
        glm_mix_pai_pei_results_df = glm_mix_pai_pei_results_df[(glm_mix_pai_pei_results_df['block_scheme'] == chosen_block_scheme) &
                                                                (glm_mix_pai_pei_results_df['model'] == chosen_model) & 
                                                                (glm_mix_pai_pei_results_df['crime_type'] == chosen_crime_type) &
                                                                (glm_mix_pai_pei_results_df['time_period'] == chosen_time_period) &
                                                                (glm_mix_pai_pei_results_df['n'] <= max_n)]
        
        lgcp_pai_pei_results_df = pd.read_csv(results_dir_path / f'lgcp_pai_pei_df_{lgcp_tag}.csv')
        lgcp_pai_pei_results_df = lgcp_pai_pei_results_df[(lgcp_pai_pei_results_df['model']==chosen_model) &
                                                          (lgcp_pai_pei_results_df['crime_type']==chosen_crime_type) &
                                                          (lgcp_pai_pei_results_df['time_period']==chosen_time_period) &
                                                          (lgcp_pai_pei_results_df['n'] <= max_n)]
        lgcp_pai_pei_results_df = lgcp_pai_pei_results_df.set_index(['n'])

        final_output =  glm_mix_pai_pei_results_df[['k', 'n', chosen_statistic]].set_index(['k', 'n']).unstack(['k'])
        final_output[chosen_statistic, 'LGCP'] = lgcp_pai_pei_results_df[chosen_statistic].values
        final_output.columns = final_output.columns.droplevel()
        final_output.to_csv(paper_output_dir_path / f'{chosen_statistic}_{chosen_crime_type}_{chosen_model}_{chosen_time_period}.dat', sep='\t', index=True)

# Block-size analysis

In [ ]:
S = 5_000
c_type = 'burglary'
model_spec = 'burglary_raw_4'

left_min = 0.05
right_min = 0.95

block_type_color_map = {'msoa': 'CustomBlue', 
                        'lad': 'CustomOrange', 
                        'single': 'CustomGreen'}

for t_period in ['12015-122015', '12013-122015']:
    full_plot_str = r"% This is a computer-generated file. DO NOT EDIT."
    full_plot_str += tex_fig_standalone_header
    full_plot_str += f"""\\begin{{tikzpicture}}%
\\begin{{axis}}[width=0.49\\textwidth, title={{Held-out log-likelihood}}, xlabel={{$K$}},xmin=0.6, xmax=8.4, xtick={{1,...,8}}, boxplot/draw direction=y]\n"""
    for block_scheme_idx, block_scheme in enumerate(['msoa', 'lad', 'single']):
        for K in [1,2,3,4,5,6,7,8]: 
            uid = build_block_mixture_flat_uid(prefix=blockmix_uid_prefix, chain_no="1", block_scheme=block_scheme,
                                                 c_type=c_type, t_period=t_period, model_spec=model_spec, resolution=400, K=K)

            context, K = get_model_description(uid)
            model = FlatRegionMixture(uid=uid, grid_context=context, K=K)

            iteration_no = get_latest_snapshot_iteration(uid)
            samples_tuple = model.load_samples_snapshot(iteration_no)

            beta_flat_samples = samples_tuple[0][-S:, :]
            Z_flat_samples = samples_tuple[1][-S:, :]

            S = min(beta_flat_samples.shape[0], Z_flat_samples.shape[0])
            print(f"Number of samples: {S}, UID: {uid}")

            beta_matrix_samples = beta_flat_samples.reshape((S, context.J, model.K), order='F')
            Z_matrix_samples = np.zeros((S, context.N, K), dtype=bool)
            Z_matrix_samples[np.repeat(range(S), context.N),
                             np.tile(range(context.N), S),
                             Z_flat_samples.flatten(order='C')] = 1

            counts_predicted_samples = compute_mixture_predicted_counts_all_samples(Z_matrix_samples,
                                                                                    beta_matrix_samples,
                                                                                    context.covariates)
            metric_all = compute_avg_loglik(counts_predicted_samples, context.test_counts)
            full_plot_str += f"\\addplot[thick, {block_type_color_map[block_scheme]}, boxplot prepared={{draw position={K - 0.2 + block_scheme_idx * 0.2}, box extend=0.15, lower whisker={np.min(metric_all)},lower quartile={np.quantile(metric_all, left_min)},median={np.median(metric_all)},upper quartile={np.quantile(metric_all, right_min)},upper whisker={np.max(metric_all)}}}] coordinates{{}};\n"
    full_plot_str += f"""\end{{axis}}%
\end{{tikzpicture}}%"""
    full_plot_str += tex_fig_standalone_footer
    with open(paper_output_dir_path / f'blocksensitivity_loglikout_{c_type}_{t_period}_{model_spec}.tex', "w") as text_file:
        print(full_plot_str, file=text_file)

In [ ]:
S = 5_000
c_type = 'burglary'
model_spec = 'burglary_raw_4'

left_min = 0.05
right_min = 0.95

block_type_color_map = {'msoa': 'CustomBlue', 
                        'lad': 'CustomOrange', 
                        'single': 'CustomGreen'}

for t_period in ['12015-122015', '12013-122015']:
    full_plot_str = r"% This is a computer-generated file. DO NOT EDIT."
    full_plot_str += tex_fig_standalone_header
    full_plot_str += f"""\\begin{{tikzpicture}}%
\\begin{{semilogyaxis}}[width=0.49\\textwidth, title={{RMSE}}, xlabel={{$K$}},xmin=0.6, xmax=8.4, xtick={{1,...,8}}, boxplot/draw direction=y]\n"""
    for block_scheme_idx, block_scheme in enumerate(['msoa', 'lad', 'single']):
        for K in [1,2,3,4,5,6,7,8]: 
            uid = build_block_mixture_flat_uid(prefix=blockmix_uid_prefix, chain_no="1", block_scheme=block_scheme,
                                                 c_type=c_type, t_period=t_period, model_spec=model_spec, resolution=400, K=K)

            context, K = get_model_description(uid)
            model = FlatRegionMixture(uid=uid, grid_context=context, K=K)

            iteration_no = get_latest_snapshot_iteration(uid)
            samples_tuple = model.load_samples_snapshot(iteration_no)

            beta_flat_samples = samples_tuple[0][-S:, :]
            Z_flat_samples = samples_tuple[1][-S:, :]

            S = min(beta_flat_samples.shape[0], Z_flat_samples.shape[0])
            print(f"Number of samples: {S}, UID: {uid}")

            beta_matrix_samples = beta_flat_samples.reshape((S, context.J, model.K), order='F')
            Z_matrix_samples = np.zeros((S, context.N, K), dtype=bool)
            Z_matrix_samples[np.repeat(range(S), context.N),
                             np.tile(range(context.N), S),
                             Z_flat_samples.flatten(order='C')] = 1

            counts_predicted_samples = compute_mixture_predicted_counts_all_samples(Z_matrix_samples,
                                                                                    beta_matrix_samples,
                                                                                    context.covariates)
            metric_all = compute_rmses_for_samples(counts_predicted_samples, context.test_counts)
            full_plot_str += f"\\addplot[thick, {block_type_color_map[block_scheme]}, boxplot prepared={{draw position={K - 0.2 + block_scheme_idx * 0.2}, box extend=0.15, lower whisker={np.min(metric_all)},lower quartile={np.quantile(metric_all, left_min)},median={np.median(metric_all)},upper quartile={np.quantile(metric_all, right_min)},upper whisker={np.max(metric_all)}}}] coordinates{{}};\n"
    full_plot_str += f"""\end{{semilogyaxis}}%
\end{{tikzpicture}}%"""
    with open(paper_output_dir_path / f'blocksensitivity_rmseout_{c_type}_{t_period}_{model_spec}.tex', "w") as text_file:
        print(full_plot_str, file=text_file)

## RMSE t-test

In [ ]:
t_period = '12015-122015'
model_spec = 'burglary_raw_4'
# K = 3

S = 5_000

for K in [2,3,4,5,6,7,8]:

    set_1_uid = build_block_mixture_flat_uid(prefix=blockmix_uid_prefix, chain_no="1", block_scheme="msoa",
                                             c_type="burglary", t_period=t_period, model_spec=model_spec, resolution=400, K=K)

    set_2_uid = build_block_mixture_flat_uid(prefix=blockmix_uid_prefix, chain_no="1", block_scheme="single",
                                             c_type="burglary", t_period=t_period, model_spec=model_spec, resolution=400, K=K)

    set_3_uid = build_block_mixture_flat_uid(prefix=blockmix_uid_prefix, chain_no="1", block_scheme="lad",
                                             c_type="burglary", t_period=t_period, model_spec=model_spec, resolution=400, K=K)

    results_1 = process_mixture_based_model((FlatRegionMixture, set_1_uid, S))['RMSE_OUT']
    results_2 = process_mixture_based_model((FlatRegionMixture, set_2_uid, S))['RMSE_OUT']
    results_3 = process_mixture_based_model((FlatRegionMixture, set_3_uid, S))['RMSE_OUT']
    
    
    print(f"K = {K}")
    
    print("MSOA VS SINGLE")
    t_stat, p_val = compute_t_test(results_1, results_2)
    print(f"M1: {np.mean(results_1)}, S1: {np.std(results_1)}, M2: {np.mean(results_2)}, S2: {np.std(results_2)}")
    print(f"t-statistic {t_stat}, P-value: {p_val}")

    print(f"MSOA VS LAD")
    t_stat, p_val = compute_t_test(results_1, results_3)
    print(f"M1: {np.mean(results_1)}, S1: {np.std(results_1)}, M2: {np.mean(results_3)}, S2: {np.std(results_3)}")
    print(f"t-statistic {t_stat}, P-value: {p_val}")


    plt.plot(results_1)
    plt.plot(results_2)
    plt.plot(results_3)
    plt.show()

# Covariate effect analysis

## LGCP

In [ ]:
lgcp_covimp_mean = pickle.load(open(results_dir_path / f"lgcp_cov_imp_mean_{lgcp_results_analysis_tag}.pickle", "rb" ))
lgcp_covimp_sd = pickle.load(open(results_dir_path / f"lgcp_cov_imp_sd_{lgcp_results_analysis_tag}.pickle", "rb" ))
lgcp_covimp_sign = pickle.load(open(results_dir_path / f"lgcp_cov_imp_sign_{lgcp_results_analysis_tag}.pickle", "rb" ))

chosen_time_period = '12013-122015'
chosen_model = "burglary_raw_4"
chosen_crime_type = 'burglary'
resolution = 400
chain_no = "2"

###### LGCP
lgcp_uid = build_lgcp_uid(prefix=lgcp_uid_prefix, chain_no=chain_no, c_type=chosen_crime_type, 
                          t_period=chosen_time_period, model_spec=chosen_model, resolution=resolution)
lgcp_context = get_model_description(lgcp_uid)
lgcp_max_iter = get_latest_snapshot_iteration(lgcp_uid)
lgcp_model = LgcpMatern(uid=lgcp_uid, grid_context=lgcp_context)
lgcp_hmc_samples = lgcp_model.load_samples_snapshot(lgcp_max_iter)
lgcp_S = min(10_000, lgcp_hmc_samples.shape[0])
NN = lgcp_context.mask.shape[0]

beta_signs = np.sign(np.mean(lgcp_hmc_samples[-lgcp_S:, (NN):(NN + lgcp_context.J)], axis=0))
beta_signs = np.append(beta_signs, 0)

temp = lgcp_covimp_mean[lgcp_uid]['1']
temp.index = [pretty_string(_cov_name) for _cov_name in temp.index]
significance_df = temp.to_frame()
significance_df.columns = ['cov_effect']
significance_df['effect_sd'] = lgcp_covimp_sd[lgcp_uid]['1'].values
significance_df['sign'] = beta_signs
significance_df['sign'] = significance_df['sign'].apply(lambda x: '-' if x < 0 else '+')
significance_df = significance_df.round({"cov_effect": 3, 'effect_sd': 3})
significance_df = significance_df.sort_values(by='cov_effect', ascending=False)
significance_latex = significance_df.to_latex()
significance_df

## BLOCK MIXTURE

In [ ]:
chosen_time_period = "12013-122015"
chosen_model = "burglary_raw_4"
chosen_K = 3
chosen_crime_type = 'burglary'
chosen_block_scheme = 'msoa'


cov_imp_means = pickle.load(open(results_dir_path / f"block_mixtures_cov_imp_mean_{blockmix_results_analysis_tag}.pickle", "rb" ))
cov_imp_sds = pickle.load(open(results_dir_path / f"block_mixtures_cov_imp_sd_{blockmix_results_analysis_tag}.pickle", "rb" ))
cov_imp_signs =  pickle.load(open(results_dir_path / f"block_mixtures_cov_imp_sign_{blockmix_results_analysis_tag}.pickle", "rb" ))


chosen_model_uid = build_block_mixture_flat_uid(prefix=blockmix_uid_prefix, chain_no="1", block_scheme=chosen_block_scheme,
                                               c_type=chosen_crime_type, t_period=chosen_time_period, model_spec=chosen_model, 
                                                resolution=400, K=chosen_K)


covariate_significance_table = cov_imp_means[chosen_model_uid]
effects_signs_table = cov_imp_signs[chosen_model_uid]

print(covariate_significance_table)
print(effects_signs_table)


context, K = get_model_description(chosen_model_uid)
max_iter = get_latest_snapshot_iteration(chosen_model_uid)
print(f"Max iter: {max_iter}")
model = FlatRegionMixture(uid=chosen_model_uid, grid_context=context, K=K)
samples_tuple = model.load_samples_snapshot(max_iter)
model.Z_samples = samples_tuple[1]

S = model.Z_samples.shape[0]


for k in range(covariate_significance_table.shape[1]):
    
    beta_signs = np.sign(np.mean(samples_tuple[0][:S, (k * context.J):((k + 1) * context.J)], axis=0))
    cov_mean_signs = np.sign(effects_signs_table.iloc[:, k].values)
    sign_product = np.multiply(beta_signs, cov_mean_signs)
    
    print(np.mean(samples_tuple[0][:S, (k * context.J):((k + 1) * context.J)], axis=0))
    print(effects_signs_table.iloc[:, k].values)
    print(context.covariates_names)
    
    temp = covariate_significance_table[k+1]
    temp.index = [pretty_string(_cov_name) for _cov_name in temp.index]
    
    significance_df = temp.to_frame()
    significance_df.columns = ['cov_effect']
    significance_df['effect_sd'] = cov_imp_sds[chosen_model_uid][k+1].values
    significance_df['sign'] = sign_product
    significance_df['sign'] = significance_df['sign'].apply(lambda x: '-' if x < 0 else '+')
    significance_df = significance_df.round({"cov_effect": 3, "effect_sd": 3})
    significance_df = significance_df.sort_values(by='cov_effect', ascending=False)
    significance_latex = significance_df.to_latex()
    print(significance_latex)

# Plots    

## Raw crime count plots

In [ ]:
crime_type = 'burglary'
time_period = '12015-122015'
uid = build_block_mixture_flat_uid(prefix=blockmix_uid_prefix, chain_no="1", block_scheme="msoa",
                                   c_type="burglary", t_period="12015-122015", model_spec="burglary_raw_4", 
                                                resolution=400, K=1)
context, K = get_model_description(uid)

training_set_crime_counts = context.counts
test_set_crime_counts = context.test_counts

# Log counts of test data
crime_gdf = gpd.GeoDataFrame(pd.DataFrame(data=np.log(1 + test_set_crime_counts), columns=[crime_type]))
crime_gdf = crime_gdf.set_geometry(context.sp_geom)
crime_gdf = crime_gdf.to_crs(epsg=3857) # required for plotting
fig, ax = plt.subplots(figsize=(0.5 * A4_INCHES_X, (1/5) * A4_INCHES_Y))
crime_gdf.plot(column=crime_type, ax=ax, legend=True, cmap='plasma', linewidth=0.0, alpha=0.7, vmin=0.0, vmax=6)
ax.set_title(f"log {crime_type.replace('.','_')}") 
ax.set_axis_off()
plt.savefig(Path(os.getcwd()) / 'reports' / 'paper_plots'/ f'crime_map_testset_log_{crime_type.replace(".", "_")}_{time_period}_{resolution}.{global_plot_format}', dpi=210, bbox_inches="tight")

# Raw counts of training data            
crime_gdf = gpd.GeoDataFrame(pd.DataFrame(data=training_set_crime_counts, columns=[crime_type]))
crime_gdf = crime_gdf.set_geometry(context.sp_geom)
crime_gdf = crime_gdf.to_crs(epsg=3857) # required for plotting            
fig, ax = plt.subplots(figsize=(0.5 * A4_INCHES_X, (1/5) * A4_INCHES_Y))
crime_gdf.plot(column=crime_type, ax=ax, legend=True, cmap='plasma', linewidth=0.0, alpha=0.7, vmin=0.0)
ax.set_title(f"log {crime_type.replace('.','_')}") 
ax.set_axis_off()
plt.savefig(Path(os.getcwd()) / 'reports' / 'paper_plots'/ f'crime_map_{crime_type.replace(".", "_")}_{time_period}_{resolution}.{global_plot_format}', dpi=210, bbox_inches="tight")

## LGCP Plots

In [ ]:
set_crime_types = ['burglary']
set_model_names = ['burglary_raw_4']
set_time_periods = ['12015-122015']
set_resolutions = ["400"]
set_chain_nos = ["1"]

### Results index to be iterated over the results
results_idx = pd.MultiIndex.from_product([set_crime_types, set_resolutions, set_model_names, set_time_periods, set_chain_nos],
                                 names=['crime_type', 'resolution', 'model', 'time_period', 'chain_no'])

for (crime_type, resolution, model_name, time_period, chain_no) in results_idx:
    
    try:
        lgcp_uid = build_lgcp_uid(prefix=lgcp_uid_prefix, chain_no=chain_no, c_type=crime_type, t_period=time_period, model_spec=model_name, resolution=resolution)
        lgcp_context = get_model_description(lgcp_uid)
        lgcp_max_iter = get_latest_snapshot_iteration(lgcp_uid)
        lgcp_model = LgcpMatern(uid=lgcp_uid, grid_context=lgcp_context)
        lgcp_hmc_samples = lgcp_model.load_samples_snapshot(lgcp_max_iter)
        lgcp_S = min(5_000, lgcp_hmc_samples.shape[0])
        NN = lgcp_context.mask.shape[0]
        
        # Plot inferred intensity LGCP
        lgcp_betas_avg = np.mean(lgcp_hmc_samples[-lgcp_S:, (NN):(NN + lgcp_context.J)], axis=0)
        lgcp_f_avg = np.mean(lgcp_hmc_samples[-lgcp_S:, :NN], axis=0)
        total_logintensity = (np.dot(lgcp_context.covariates, lgcp_betas_avg) + lgcp_f_avg[lgcp_context.mask])
        
        overdispersion_plot_data_in = pd.DataFrame({'predicted': np.exp(total_logintensity), 'realised': context.counts})
        overdispersion_plot_data_in.to_csv(paper_output_dir_path / f'predicted_vs_realised_in_lgcp_{crime_type}_{model_name}_{time_period}.dat', sep='\t', index=False)      
        
        overdispersion_plot_data_out = pd.DataFrame({'predicted': np.exp(total_logintensity), 'realised': context.test_counts})
        overdispersion_plot_data_out.to_csv(paper_output_dir_path / f'predicted_vs_realised_out_lgcp_{crime_type}_{model_name}_{time_period}.dat', sep='\t', index=False)      
        chi_sq = np.sum(np.divide(np.square(np.exp(total_logintensity) - context.test_counts), np.exp(total_logintensity)))
        
        
        total_intensity_gdf = gpd.GeoDataFrame(pd.DataFrame(data=np.log(1 + np.exp(total_logintensity)), columns=['mean_intensity']))
        total_intensity_gdf = total_intensity_gdf.set_geometry(lgcp_context.sp_geom)
        total_intensity_gdf = total_intensity_gdf.to_crs(epsg=3857) # required for plotting
        
        fig, ax = plt.subplots(figsize=(0.5 * A4_INCHES_X, (1/5) * A4_INCHES_Y))
        total_intensity_gdf.plot(column='mean_intensity', ax=ax, legend=True, cmap='plasma', linewidth=0.0, alpha=0.7, vmin=0.0, vmax=6)
        ax.set_title(f"Mean predicted log intensity (LGCP)")
        ax.set_axis_off()
        fpath = paper_output_dir_path/ f'lgcp_mean_intensity_chain_{chain_no}_{crime_type}_{time_period}_{model_name}_{resolution}.{global_plot_format}'
        plt.savefig(fpath, dpi=210, bbox_inches="tight")
        plt.show()
    
        ## Plot the latent field for the LGCP
        sns.set_context("paper", font_scale=1.0, rc={"lines.linewidth":1.0})
        field_samples = lgcp_hmc_samples[:, :NN]
        field_on_map_samples = field_samples[:, lgcp_context.mask]
        
        gdf = gpd.GeoDataFrame(pd.DataFrame({'mean': np.mean(field_on_map_samples, axis=0),'sd': np.std(field_on_map_samples, axis=0)}))
        gdf = gdf.set_geometry(lgcp_context.sp_geom)
        gdf = gdf.to_crs(epsg=3857) # required for plotting

        fig, ax = plt.subplots(figsize=(0.5 * A4_INCHES_X, (1/5) * A4_INCHES_Y))
        ax.set_title("$f$ mean")
        gdf.plot(column='mean', ax=ax,
                 legend=True, cmap='plasma',
                 linewidth=0.0, alpha=0.6)
        ax.set_axis_off()
        fpath = paper_output_dir_path / f'lgcp_field_mean_chain_{chain_no}_{crime_type}_{time_period}_{model_name}_{resolution}.{global_plot_format}'
        plt.savefig(fpath, dpi=210, bbox_inches="tight")
        plt.show()

        fig, ax = plt.subplots(figsize=(0.5 * A4_INCHES_X, (1/5) * A4_INCHES_Y))
        ax.set_title("$f$ standard deviation")
        gdf.plot(column='sd', ax=ax,
                 legend=True, cmap='plasma', vmin=0,
                 linewidth=0.0, alpha=0.6)
        ax.set_axis_off()
        fpath = paper_output_dir_path / f'lgcp_field_sd_chain_{chain_no}_{crime_type}_{time_period}_{model_name}_{resolution}.{global_plot_format}'
        plt.savefig(fpath, dpi=210, bbox_inches="tight")
        plt.show()
        
        ## Visualise the hyper-parameters
        hpar_samples = np.exp(lgcp_hmc_samples[:, (NN + lgcp_context.J):])
        hpar_samples_df = pd.DataFrame(hpar_samples)
        hpar_samples_df.columns = ['variance', 'lengthscale']
        
        sns.set_context("paper", font_scale=1.0, rc={"lines.linewidth":1.0})
        fig, ax = plt.subplots(figsize=(.5*A4_INCHES_X, (1/5) * A4_INCHES_Y))
        for col_name in hpar_samples_df.columns:
            sns.distplot(hpar_samples_df[col_name], ax=ax, kde=False)
        
        sns.despine(offset=1, trim=False)
        plt.legend(['variance', 'lengthscale'])
        plt.xlabel("")
        fpath = paper_output_dir_path / f'lgcp_hyperplot_chain_{chain_no}_{crime_type}_{time_period}_{model_name}_{resolution}.{global_plot_format}'
        plt.savefig(fpath, dpi=210, bbox_inches="tight")
        plt.show()

    except FileNotFoundError as e:
        print(f"No result for model: {model_name}")
        print(e)
        continue

## SAM-GLM plots

In [ ]:
set_crime_types = ['burglary']
set_model_names = ['burglary_raw_4']
set_Ks = [1, 3]
set_block_schemes = ["msoa"]
set_time_periods = ['12015-122015']
set_resolutions = ["400"]

set_chain_nos = ["1"]

### Results index to be iterated over the results
results_idx = pd.MultiIndex.from_product([set_crime_types, set_resolutions, set_model_names, set_time_periods, set_block_schemes, set_Ks, set_chain_nos],
                                 names=['crime_type', 'resolution', 'model', 'time_period', 'block_scheme', 'k', 'chain_no'])


for (crime_type, resolution, model_name, time_period, block_scheme, num_mixtures, chain_no) in results_idx:
    
    uid = build_block_mixture_flat_uid(prefix=blockmix_uid_prefix, chain_no=chain_no, block_scheme=block_scheme,
                                               c_type=crime_type, t_period=time_period, model_spec=model_name, 
                                                resolution=resolution, K=num_mixtures)

    print(uid)
    try:
        context, K = get_model_description(uid)
        max_iter = get_latest_snapshot_iteration(uid)
        print(f"Max iter: {max_iter}")
        model = FlatRegionMixture(uid=uid, grid_context=context, K=K)

        samples_tuple = model.load_samples_snapshot(max_iter)
        model.Z_samples = samples_tuple[1]

        S = min(5_000, samples_tuple[0].shape[0])
        N = context.N
        K = num_mixtures
        J = context.J
        
        beta_samples = samples_tuple[0]
        beta_samples = beta_samples[-S:, :]
        z_samples = np.asarray(samples_tuple[1])
        z_samples = z_samples[-S:, :]

        mixture_allocation = np.zeros((S, N, K), dtype=bool)
        mixture_allocation[np.repeat(range(S), N), np.tile(range(N), S), z_samples.flatten(order='C')] = 1
      
        average_alloc = np.mean(mixture_allocation, axis=0)
        average_beta_matrix = np.mean(beta_samples, axis=0).reshape((J, K), order='F')
        
        # Print the proportions of crimes covered by the components
        z_k_mean = np.argmax(average_alloc, axis=1)
        z_k_mean_binary = np.zeros((N, K), dtype=bool)
        z_k_mean_binary[range(N), z_k_mean] = 1
        print(f"The proportion of crimes covered by components: {np.sum((z_k_mean_binary * context.counts[:, np.newaxis]) / (np.sum(context.counts)), axis=0)}")
        
        ### Plot the allocation maps and log-intensities ###
        intensity_gdf = gpd.GeoDataFrame(pd.DataFrame(data=np.exp(np.multiply(average_alloc, np.dot(context.covariates, average_beta_matrix))),
                                                          columns=np.arange(num_mixtures) + 1))
        intensity_gdf = intensity_gdf.set_geometry(context.sp_geom)
        intensity_gdf = intensity_gdf.to_crs(epsg=3857) # required for plotting
        
        allocation_gdf = gpd.GeoDataFrame(pd.DataFrame(data=average_alloc, columns=np.arange(num_mixtures)+1))
        allocation_gdf = allocation_gdf.set_geometry(context.sp_geom)
        allocation_gdf = allocation_gdf.to_crs(epsg=3857) # required for plotting
        
        # Plot inferred intensity
        total_intensity = np.sum(np.multiply(average_alloc, np.dot(context.covariates, average_beta_matrix)), axis=1)
        total_intensity_gdf = gpd.GeoDataFrame(pd.DataFrame(data=np.log(1 + np.exp(total_intensity)), columns=['mean_intensity']))
        total_intensity_gdf = total_intensity_gdf.set_geometry(context.sp_geom)
        total_intensity_gdf = total_intensity_gdf.to_crs(epsg=3857) # required for plotting
        
        
        
        # generate data for scatter plots
        overdispersion_plot_data_in = pd.DataFrame({'predicted': np.exp(total_intensity), 'realised': context.counts})
        overdispersion_plot_data_in.to_csv(paper_output_dir_path / f'predicted_vs_realised_in_{crime_type}_{block_scheme}_{model_name}_{time_period}_{K}.dat', sep='\t', index=False)
        fig, ax = plt.subplots(figsize=(0.5 * A4_INCHES_X, 0.5 * A4_INCHES_X))
        ax.scatter(overdispersion_plot_data_in['predicted'], 
                   overdispersion_plot_data_in['realised'],
                   color='gray', alpha=0.5)
        chi_sq = np.sum(np.divide(np.square(np.exp(total_intensity) - context.counts), total_intensity))
        ax.text(0.15*np.max(context.counts), 0.95*np.max(context.counts), f"Pearson $\chi^2$ = {round(chi_sq, 2)}", fontsize=12)
        lims = [
            np.min([ax.get_xlim(), ax.get_ylim()]),  # min of both axes
            np.max([ax.get_xlim(), ax.get_ylim()]),  # max of both axes
        ]
        # now plot both limits against eachother
        ax.plot(lims, lims, 'k-', alpha=0.75, zorder=100)
        ax.set_aspect('equal')
        ax.set_xlim(lims)
        ax.set_ylim(lims)
        plt.xlabel("Expected")
        plt.ylabel("Observed")
        fpath = paper_output_dir_path / f'predicted_vs_realised_in_{crime_type}_{block_scheme}_{model_name}_{time_period}_{K}.{global_plot_format}'
        plt.savefig(fpath)
        plt.show()
        
        fig, ax = plt.subplots(figsize=(0.5 * A4_INCHES_X, (1/5) * A4_INCHES_Y))
        total_intensity_gdf.plot(column='mean_intensity', ax=ax, legend=True, cmap='plasma', linewidth=0.0, alpha=0.7, vmin=0.0, vmax=6)
        ax.set_title(f"Mean predicted log intensity (Mixture, K={num_mixtures})")
        ax.set_axis_off()
        fpath = Path(os.getcwd()) / 'reports' / 'paper_plots'/ f'mixtures_mean_intensity_chain_{chain_no}_{block_scheme}_{crime_type}_{time_period}_{model_name}_{resolution}_{num_mixtures}.{global_plot_format}'
        plt.savefig(fpath, dpi=210, bbox_inches="tight")
        plt.show()
        
        # Plot components wise
        for k in range(num_mixtures):
            fig, ax = plt.subplots(figsize=(0.5 * A4_INCHES_X, (1/5) * A4_INCHES_Y))
            allocation_gdf.plot(column=k+1, ax=ax,
                     legend=True, cmap='plasma', linewidth=0.0, alpha=0.7)
            ax.set_title(f"Allocation, component {k+1}")
            ax.set_axis_off()
            fpath = Path(os.getcwd()) / 'reports' / 'paper_plots'/ f'mixtures_allocation_chain_{chain_no}_{block_scheme}_{crime_type}_{time_period}_{model_name}_{resolution}_{num_mixtures}_{k}.{global_plot_format}'
            plt.savefig(fpath, dpi=210, bbox_inches="tight")
            plt.show()
            
            fig, ax = plt.subplots(figsize=(0.5 * A4_INCHES_X, (1/5) * A4_INCHES_Y))
            intensity_gdf.plot(column=k+1, ax=ax,
                     legend=True, cmap='plasma', linewidth=0.0, alpha=0.7)
            ax.set_title(f"Intensity, component {k+1}")
            ax.set_axis_off()
            fpath = Path(os.getcwd()) / 'reports' / 'paper_plots'/ f'mixtures_intensity_chain_{chain_no}_{block_scheme}_{crime_type}_{time_period}_{model_name}_{resolution}_{num_mixtures}_{k}.{global_plot_format}'
            plt.savefig(fpath, dpi=210, bbox_inches="tight")
            plt.show()
        
    except FileNotFoundError as e:
        print(f"No result for region: {block_scheme}, K={num_mixtures}, model: {model_name}")
        continue

# Diagnostics

## BLOCK MIXTURE

In [ ]:
crime_type = 'burglary'
resolution = 400
model_name = 'burglary_raw_4'
time_period = '12015-122015'
block_scheme = 'msoa'
num_mixtures = 3
num_samples = 5_000

uid = build_block_mixture_flat_uid(prefix=blockmix_uid_prefix, chain_no="1", block_scheme=block_scheme,
                                       c_type=crime_type, t_period=time_period, model_spec=model_name, resolution=resolution, K=num_mixtures)
print(uid)
context, K = get_model_description(uid)
max_iter = get_latest_snapshot_iteration(uid)
print(f"Max iter: {max_iter}")
model = FlatRegionMixture(uid=uid, grid_context=context, K=K)

samples_tuple = model.load_samples_snapshot(max_iter)
model.Z_samples = samples_tuple[1]

S = min(num_samples, samples_tuple[0].shape[0])
N = context.N
K = num_mixtures
J = context.J

beta_samples = samples_tuple[0]
beta_samples = beta_samples[-S:, :]
z_samples = np.asarray(samples_tuple[1])
z_samples = z_samples[-S:, :]

mixture_allocation = np.zeros((S, N, K), dtype=bool)
mixture_allocation[np.repeat(range(S), N), np.tile(range(N), S), z_samples.flatten(order='C')] = 1

# likelihood plot
ind_blocks_log_lik_samples_in = np.zeros(S)
ind_blocks_log_lik_samples_out = np.zeros(S)
for s in range(S):
    beta = beta_samples[s, :]
    beta_matrix = beta.reshape((J, K), order='F')

    current_Z = mixture_allocation[s, :, :]

    fixed_effects = np.sum(np.multiply(current_Z, np.dot(context.covariates, beta_matrix)), axis=1)
    ind_blocks_log_lik_samples_in[s] = np.mean(poisson.logpmf(context.counts, np.exp(fixed_effects)))
    ind_blocks_log_lik_samples_out[s] = np.mean(poisson.logpmf(context.test_counts, np.exp(fixed_effects)))

plot_traceplots(ind_blocks_log_lik_samples_in.reshape((S, -1)), ['likelihood'])
plt.show()

plt.figure(figsize=(0.5*A4_INCHES_X, 0.5*A4_INCHES_X))
ax = plt.subplot(111)
plot_acf(ind_blocks_log_lik_samples_in, title='Independent blocks', ax=ax, zero=False)
ax.yaxis.set_major_formatter(FormatStrFormatter('%.3f'))
plt.xlabel('Lag')
plt.ylabel('ACF')
fpath = paper_output_dir_path / f'blockmix_acf_{block_scheme}_{crime_type}_{time_period}_{model_name}_{resolution}.{global_plot_format}'
plt.savefig(fpath)
plt.show()

for k in range(K):
    beta_k_samples = beta_samples[:, (k * J):((k + 1) * J)]
    beta_k_plot = plot_traceplots(beta_k_samples, context.covariates_names)
    plt.show()

Below, we also plot the histogram of the samples of RMSE and log-likelihood for both variants of the block mixtures
model and for both in-sample and out-of-sample data.

In [ ]:
from scipy import stats

density_ind = stats.gaussian_kde(ind_blocks_log_lik_samples_in)
density_dep = stats.gaussian_kde(dep_blocks_log_lik_samples_in)

plt.figure(figsize=(0.5*A4_INCHES_X, 0.5*A4_INCHES_X))
d_min = min(np.min(ind_blocks_log_lik_samples_in), np.min(dep_blocks_log_lik_samples_in))
d_max = max(np.max(ind_blocks_log_lik_samples_in), np.max(dep_blocks_log_lik_samples_in))
xs = np.linspace(d_min, d_max, 100)
ax = plt.subplot(111)
ax.plot(xs,density_ind(xs))
ax.plot(xs,density_dep(xs))
fpath = paper_output_dir_path / f'ind_vs_dep_loglik_in_{block_scheme}_{crime_type}_{time_period}_{model_name}_{resolution}.{global_plot_format}'
plt.title("In-sample log-likelihood")
plt.savefig(fpath, bbox_inches="tight")
plt.show()

density_ind = stats.gaussian_kde(ind_blocks_log_lik_samples_out)
density_dep = stats.gaussian_kde(dep_blocks_log_lik_samples_out)

plt.figure(figsize=(0.5*A4_INCHES_X, 0.5*A4_INCHES_X))
d_min = min(np.min(ind_blocks_log_lik_samples_out), np.min(dep_blocks_log_lik_samples_out))
d_max = max(np.max(ind_blocks_log_lik_samples_out), np.max(dep_blocks_log_lik_samples_out))
xs = np.linspace(d_min, d_max, 100)
ax = plt.subplot(111)
ax.plot(xs,density_ind(xs))
ax.plot(xs,density_dep(xs))
plt.title("Held-out log-likelihood")
fpath = paper_output_dir_path / f'ind_vs_dep_loglik_out_{block_scheme}_{crime_type}_{time_period}_{model_name}_{resolution}.{global_plot_format}'
plt.savefig(fpath)
plt.show()



## BLOCK MIXTURE GP

In [ ]:
model_name = 'burglary_raw_4'
time_period = '12015-122015'
num_mixtures = 3
lengthscale = 1_500
block_scheme = 'msoa'
resolution = 400
crime_type = 'burglary'
num_samples=5_000

uid = build_block_mixture_gp_softmax_uid(prefix=blockgp_softmax_uid_prefix, 
                                         chain_no=None, block_scheme=block_scheme,
                                           c_type=crime_type, t_period=time_period, model_spec=model_name,
                                           resolution=resolution, K=num_mixtures, lengthscale=lengthscale)

context, K = get_model_description(uid)
max_iter = get_latest_snapshot_iteration(uid)
model = BlockMixtureGpSoftmaxAllocation(uid=uid, grid_context=context, K=K)

samples_tuple = model.load_samples_snapshot(max_iter)
model.Z_samples = samples_tuple[1]

S = min(num_samples, samples_tuple[0].shape[0])
N = context.N
K = num_mixtures
J = context.J
beta_samples = samples_tuple[0]
beta_samples = beta_samples[-S:, :]
Z_samples = np.asarray(samples_tuple[1])
Z_samples = Z_samples[-S:, :]
F_samples = np.asarray(samples_tuple[2])

f_array = np.asarray(F_samples).reshape((-1, model.B, K), order='F')
idx1 = np.random.choice(model.B)
plot_traceplots(f_array[:, idx1, :], [f"IDX: {idx1}: K={k}" for k in range(model.K)])
plt.show()


mixture_allocation = np.zeros((S, N, K), dtype=bool)
mixture_allocation[np.repeat(range(S), N), np.tile(range(N), S), Z_samples.flatten(order='C')] = 1


f_loglik_samples = np.zeros(S)
for s in range(S):
    model.current_Z = mixture_allocation[s, :, :]
    f_loglik_samples[s] = model.f_loglik(F_samples[s, :])
plot_traceplots(f_loglik_samples.reshape((S, -1)), ['likelihood'])
plt.show()

# likelihood plot
dep_blocks_log_lik_samples_in = np.zeros(S)
dep_blocks_log_lik_samples_out = np.zeros(S)
for s in range(S):
    beta = beta_samples[s, :]
    beta_matrix = beta.reshape((J, K), order='F')

    current_Z = mixture_allocation[s, :, :]
    fixed_effects = np.sum(np.multiply(current_Z, np.dot(context.covariates, beta_matrix)), axis=1)
    dep_blocks_log_lik_samples_in[s] = np.mean(poisson.logpmf(context.counts, np.exp(fixed_effects)))
    dep_blocks_log_lik_samples_out[s] = np.mean(poisson.logpmf(context.test_counts, np.exp(fixed_effects)))
plot_traceplots(dep_blocks_log_lik_samples_in.reshape((S, -1)), ['log-likelihood'])
plt.show()

plt.figure(figsize=(0.5*A4_INCHES_X, 0.5*A4_INCHES_X))
ax = plt.subplot(111)
plot_acf(dep_blocks_log_lik_samples_in, title='Dependent blocks', ax=ax, zero=False)
ax.yaxis.set_major_formatter(FormatStrFormatter('%.3f'))
plt.xlabel('Lag')
plt.ylabel('ACF')
fpath = paper_output_dir_path / f'blockmix_gp_acf_{block_scheme}_{crime_type}_{time_period}_{model_name}_{resolution}.{global_plot_format}'
plt.savefig(fpath)
plt.show()

for k in range(K):
    beta_k_samples = beta_samples[:, (k * context.J):((k + 1) * context.J)]
    beta_k_plot = plot_traceplots(beta_k_samples, context.covariates_names)
    plt.show()


In [ ]:
model_name = 'burglary_raw_4'
time_period = '12015-122015'
num_mixtures = 3
lengthscale = 1_500
block_scheme = 'msoa'
resolution = 400
crime_type = 'burglary'
num_samples=5_000

uid = build_block_mixture_gp_softmax_uid(prefix=blockgp_softmax_uid_prefix, 
                                         chain_no=None, block_scheme=block_scheme,
                                           c_type=crime_type, t_period=time_period, model_spec=model_name,
                                           resolution=resolution, K=num_mixtures, lengthscale=lengthscale)

context, K = get_model_description(uid)
max_iter = get_latest_snapshot_iteration(uid)
model = BlockMixtureGpSoftmaxAllocation(uid=uid, grid_context=context, K=K)

samples_tuple = model.load_samples_snapshot(max_iter)
model.Z_samples = samples_tuple[1]

S = min(num_samples, samples_tuple[0].shape[0])
N = context.N
K = num_mixtures
J = context.J
beta_samples = samples_tuple[0]
beta_samples = beta_samples[-S:, :]
Z_samples = np.asarray(samples_tuple[1])
Z_samples = Z_samples[-S:, :]
F_samples = np.asarray(samples_tuple[2])

f_array = np.asarray(F_samples).reshape((-1, model.B, K), order='F')
idx1 = np.random.choice(model.B)
plot_traceplots(f_array[:, idx1, :], [f"IDX: {idx1}: K={k}" for k in range(model.K)])
plt.show()


mixture_allocation = np.zeros((S, N, K), dtype=bool)
mixture_allocation[np.repeat(range(S), N), np.tile(range(N), S), Z_samples.flatten(order='C')] = 1


f_loglik_samples = np.zeros(S)
for s in range(S):
    model.current_Z = mixture_allocation[s, :, :]
    f_loglik_samples[s] = model.f_loglik(F_samples[s, :])
plot_traceplots(f_loglik_samples.reshape((S, -1)), ['likelihood'])
plt.show()

# likelihood plot
dep_blocks_log_lik_samples_in = np.zeros(S)
dep_blocks_log_lik_samples_out = np.zeros(S)
for s in range(S):
    beta = beta_samples[s, :]
    beta_matrix = beta.reshape((J, K), order='F')

    current_Z = mixture_allocation[s, :, :]
    fixed_effects = np.sum(np.multiply(current_Z, np.dot(context.covariates, beta_matrix)), axis=1)
    dep_blocks_log_lik_samples_in[s] = np.mean(poisson.logpmf(context.counts, np.exp(fixed_effects)))
    dep_blocks_log_lik_samples_out[s] = np.mean(poisson.logpmf(context.test_counts, np.exp(fixed_effects)))
plot_traceplots(dep_blocks_log_lik_samples_in.reshape((S, -1)), ['log-likelihood'])
plt.show()

plt.figure(figsize=(0.5*A4_INCHES_X, 0.5*A4_INCHES_X))
ax = plt.subplot(111)
plot_acf(dep_blocks_log_lik_samples_in, title='Dependent blocks', ax=ax, zero=False)
ax.yaxis.set_major_formatter(FormatStrFormatter('%.3f'))
plt.xlabel('Lag')
plt.ylabel('ACF')
fpath = paper_output_dir_path / f'blockmix_gp_acf_{block_scheme}_{crime_type}_{time_period}_{model_name}_{resolution}.{global_plot_format}'
plt.savefig(fpath)
plt.show()

for k in range(K):
    beta_k_samples = beta_samples[:, (k * context.J):((k + 1) * context.J)]
    beta_k_plot = plot_traceplots(beta_k_samples, context.covariates_names)
    plt.show()